In [ ]:
import sqlite3
from io import StringIO
import sys

# Funções e classes mínimas simulando your_module
def init_db(conn):
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS historico_simulacoes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ganho_planejado REAL,
        custo_material REAL,
        gasto_fixo_mensal REAL,
        lucro_necessario REAL,
        investimento_reservado REAL,
        emergencia_reservada REAL,
        lucro_liquido REAL
    )''')
    conn.commit()

def inserir_historico(conn, ganho_planejado, custo_material, gasto_fixo_mensal, lucro_necessario, investimento_reservado, emergencia_reservada, lucro_liquido):
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO historico_simulacoes (ganho_planejado, custo_material, gasto_fixo_mensal, lucro_necessario, investimento_reservado, emergencia_reservada, lucro_liquido)
    VALUES (?, ?, ?, ?, ?, ?, ?)''', (ganho_planejado, custo_material, gasto_fixo_mensal, lucro_necessario, investimento_reservado, emergencia_reservada, lucro_liquido))
    conn.commit()

def ver_historico(conn):
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM historico_simulacoes ORDER BY id DESC LIMIT 1')
    resultado = cursor.fetchone()
    if resultado:
        print("\n--- Última Simulação ---")
        print(f"Ganho Planejado: R$ {resultado[1]:.2f}")
        print(f"Custo de Materiais: R$ {resultado[2]:.2f}")
        print(f"Gasto Fixo Mensal: R$ {resultado[3]:.2f}")
        print(f"Lucro Necessário: R$ {resultado[4]:.2f}")
        print(f"Investimento Reservado: R$ {resultado[5]:.2f}")
        print(f"Emergência Reservada: R$ {resultado[6]:.2f}")
        print(f"Lucro Líquido: R$ {resultado[7]:.2f}")
    else:
        print("Nenhuma simulação registrada.")

# Teste para inserir e verificar dados no histórico
def test_integracao_inserir_e_verificar_historico():
    conn = sqlite3.connect(':memory:')
    init_db(conn)

    dados_historico = {
        "ganho_planejado": 1500.00,
        "custo_material": 700.00,
        "gasto_fixo_mensal": 300.00,
        "lucro_necessario": 500.00,
        "investimento_reservado": 50.00,
        "emergencia_reservada": 30.00,
        "lucro_liquido": 420.00
    }

    inserir_historico(conn, **dados_historico)

    cursor = conn.cursor()
    cursor.execute('SELECT * FROM historico_simulacoes ORDER BY id DESC LIMIT 1')
    resultado = cursor.fetchone()

    assert resultado is not None, "Nenhum registro no histórico"
    assert abs(resultado[1] - dados_historico["ganho_planejado"]) < 1e-6
    assert abs(resultado[7] - dados_historico["lucro_liquido"]) < 1e-6

    conn.close()

# Teste para verificar a saída da função ver_historico
def test_integracao_ver_historico():
    conn = sqlite3.connect(':memory:')
    init_db(conn)
    inserir_historico(conn, 1000.00, 500.00, 200.00, 300.00, 50.00, 30.00, 220.00)

    # Captura a saída padrão
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    ver_historico(conn)

    sys.stdout = old_stdout
    output = mystdout.getvalue()

    assert "Última Simulação" in output
    assert "Lucro Líquido: R$ 220.00" in output

    conn.close()

# Executando os testes
test_integracao_inserir_e_verificar_historico()
test_integracao_ver_historico()
print("✅ Testes de integração do histórico passaram!")